In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinBiolinum_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.2)

frames = [pd.read_csv(data, names=[
    "NEAT", "_protocol", "chunk_k", "chunk_n", "chunk_m", "k", "n", "num_peer", "num_op", "op", "latency", "url", "id"
]) for data in glob('entropy-5.txt')]
data = pd.concat(frames, ignore_index=True)
data = data[data['NEAT'] == 'NEAT']
data['latency'] = data['latency'].apply(pd.to_numeric)
data['num_peer'] = data['num_peer'].apply(pd.to_numeric)
data

In [ ]:
def outer_code(row):
    return f'({row["n"]}, {row["k"]})'
data['outer_code'] = data.apply(outer_code, axis=1)
def inner_code(row):
    return f'({row["chunk_n"]}, {row["chunk_k"]})'
data['inner_code'] = data.apply(inner_code, axis=1)
def op2(row):
    return f'{row["_protocol"]} - {row["op"]}'
data['op2'] = data.apply(op2, axis=1)
data

In [ ]:
def group(row):
    if row['inner_code'] == '(80, 32)':
        return row['outer_code']
    else:
        return '(10, 8)\n' + row['inner_code']
data['group'] = data.apply(group, axis=1)
f = sns.catplot(data=data[(data["num_peer"] == 10000)], 
    kind="bar",
    x="group", y="latency", hue="op2",
    order=["(5, 4)", "(10, 8)", "(20, 16)", "(10, 8)\n(40, 16)", "(10, 8)\n(160, 64)"],
    height=2.56, aspect=2.)
f.set(xlabel = "Outer Code (and inner code on second line)")
f.set(ylabel="Latency (s)")
f._legend.set_title('Operation')
f.savefig('latency.pdf')

In [ ]:
sns.catplot(data=data[(data["outer_code"] == "(10, 8)") & (data["num_peer"] == 10000) & (data["_protocol"] == "entropy")], 
    kind="bar",
    x="inner_code", y="latency", hue="op2",
    order=["(40, 16)", "(80, 32)", "(160, 64)"],
    height=2.56, aspect=2.)

In [ ]:
f = sns.relplot(data=data[(data["outer_code"] == "(10, 8)") & (data["inner_code"] == "(80, 32)")],
    kind="line",
    x="num_peer", y="latency", hue="op2",
    height=2.56, aspect=2.)
f.set(xlabel = "Number of Peers")
f.set(ylabel="Latency (s)")
f._legend.set_title('Operation')
f.set(ylim=(0, 60))
f.savefig('peer.pdf')

In [ ]:
frames = [pd.read_csv(data, names=[
    "NEAT", "num_replica", "faulty_portion", "num_alive",
]) for data in glob('entropy-ipfs-loss.txt')]
data = pd.concat(frames, ignore_index=True)
data = data[data['NEAT'] == 'NEAT']
data['num_replica'] = data['num_replica'].apply(pd.to_numeric)
data['faulty_portion'] = data['faulty_portion'].apply(pd.to_numeric)
data['num_alive'] = data['num_alive'].apply(pd.to_numeric)
data

In [ ]:
sns.relplot(data=data,
    kind="line",
    x="faulty_portion", y="num_alive", hue="num_replica",
    height=2.56, aspect=2.)

In [ ]:
frames = [pd.read_csv(data, names=[
    "NEAT", "_protocol", "chunk_k", "chunk_n", "chunk_m", "k", "n", "num_peer", "num_op", "op", "latency", "url", "id"
]) for data in glob('entropy-1.txt')]
data = pd.concat(frames, ignore_index=True)
data = data[data['NEAT'] == 'NEAT']
data['latency'] = data['latency'].apply(pd.to_numeric)
data

In [ ]:
f = sns.relplot(data=data,
    kind="line",
    x="num_op", y="latency", hue="op",
    height=2.56, aspect=2.)